# BERT2BERT

На основе https://github.com/huggingface/notebooks/blob/master/examples/summarization.ipynb

Загрузка данных

In [ ]:
!wget https://www.dropbox.com/s/mbj3sb6jaw3d9s3/judgements_test.json
!wget https://www.dropbox.com/s/hbh7ioaiet16fcw/judgements_train.json

--2021-05-23 18:22:47--  https://www.dropbox.com/s/mbj3sb6jaw3d9s3/judgements_test.json
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/mbj3sb6jaw3d9s3/judgements_test.json [following]
--2021-05-23 18:22:48--  https://www.dropbox.com/s/raw/mbj3sb6jaw3d9s3/judgements_test.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9ee1cb6daf26ada0ba773373f1.dl.dropboxusercontent.com/cd/0/inline/BPBhjq0Ib_zhAKGwPsnkuqMZLCONbCFirXtmFy0skLHWK4UYZN0RzlenF3UlLF1Iy0XzbV9aj-NWHk1xCXLSug2BT5eiU688QAKx9Qp22Poe5-4tYo08LcfJzznHT2-iYwWiKvQ7uHBkwhtUSBBMHfA_/file# [following]
--2021-05-23 18:22:48--  https://uc9ee1cb6daf26ada0ba773373f1.dl.dropboxusercontent.com/cd/0/inline/BPBhjq0Ib_zhAKGwPsnkuqMZLCONbCFirXtmFy0skLHWK4UYZN0RzlenF3UlLF1Iy0

Установка нужных библиотек

In [ ]:
! pip install transformers datasets sentencepiece

     |████████████████████████████████| 2.3MB 35.8MB/s 
     |████████████████████████████████| 225kB 50.9MB/s 
     |████████████████████████████████| 1.2MB 49.4MB/s 
     |████████████████████████████████| 901kB 40.6MB/s 
     |████████████████████████████████| 3.3MB 50.5MB/s 
     |████████████████████████████████| 245kB 57.6MB/s 
     |████████████████████████████████| 112kB 58.7MB/s 


In [ ]:
import json

In [ ]:
with open ('judgements_train.json', encoding='utf-8') as f:
  data_train = json.load(f)

with open ('judgements_test.json', encoding='utf-8') as f:
  data_test = json.load(f)

Часть тренировочной выборки переобозначаем в валидационную:

In [ ]:
from sklearn.model_selection import train_test_split

_, val_keys = train_test_split(list(data_train.keys()), test_size=0.15, random_state=10)

data_val = {}

for k in val_keys:
  data_val[k] = data_train[k]
  data_train.pop(k)

In [ ]:
n_data_train = {'id': [], 'texts': [], 'summaries': []}

for ind in data_train.keys():
  n_data_train['id'].append(ind)
  n_data_train['texts'].append(data_train[ind]['text'])
  n_data_train['summaries'].append(data_train[ind]['summary'])

n_data_val = {'id': [], 'texts': [], 'summaries': []}

for ind in data_val.keys():
  n_data_val['id'].append(ind)
  n_data_val['texts'].append(data_val[ind]['text'])
  n_data_val['summaries'].append(data_val[ind]['summary'])

n_data_test = {'id': [], 'texts': [], 'summaries': []}

for ind in data_test.keys():
  n_data_test['id'].append(ind)
  n_data_test['texts'].append(data_test[ind]['text'])
  n_data_test['summaries'].append(data_test[ind]['summary'])

Преобразовываем словари с текстами в формат Dataset, удобный для работы с моделями huggingface

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict(n_data_train)

In [ ]:
train_dataset

Dataset({
    features: ['id', 'texts', 'summaries'],
    num_rows: 12925
})

In [ ]:
val_dataset = Dataset.from_dict(n_data_val)
test_dataset = Dataset.from_dict(n_data_test)

Инициализируем модель и токенизатор весами ruBERT, обученного на задаче генерации заголовков https://huggingface.co/dmitry-vorobiev/rubert_ria_headlines 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "dmitry-vorobiev/rubert_ria_headlines"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Подготавливаем текстовые данные для подачи в модель: слова превращаются в идентификаторы, обрезаются или дополняются нулями до 512 символов (это особенность исходной модели)

In [ ]:
max_input_length = 512
max_target_length = 100

def preprocess_function(examples):
    inputs = [doc for doc in examples["texts"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, 
                             truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summaries"],
                           max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


Указываем параметры для обучения (в данном случае, основные параметры такие же, как в исходном ноутбуке от hugging face, а batch_size довольно малый для того, чтобы все смогло влезть в память Google Colab)

In [ ]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Загрузка метрики:

In [ ]:
!pip install rouge_score

In [ ]:
from datasets import load_metric

metric = load_metric("rouge")

Функция для подсчета метрики во время обучения

In [ ]:
import nltk
import numpy as np

nltk.download('punkt')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

Процесс обучения

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.890100,1.715414,76.284800,0.746100,76.268000,76.326900,47.108200
2,1.607400,1.553113,81.318600,0.987900,81.333600,81.375800,47.569700


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.890100,1.715414,76.284800,0.746100,76.268000,76.326900,47.108200
2,1.607400,1.553113,81.318600,0.987900,81.333600,81.375800,47.569700
3,1.491700,1.484088,80.574400,1.583600,80.594000,80.625600,47.419800


KeyboardInterrupt: ignored

Считаем метрику на тестовом наборе данных:

In [ ]:
trainer.predict(tokenized_test_dataset)

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.890100,1.715414,76.284800,0.746100,76.268000,76.326900,47.108200
2,1.607400,1.553113,81.318600,0.987900,81.333600,81.375800,47.569700
3,1.491700,1.484088,80.574400,1.583600,80.594000,80.625600,47.419800


PredictionOutput(predictions=array([[   101,  29971,   1523, ...,    132,    845,  85821],
       [   101,  29971,   1523, ...,  28847,  87085,  16353],
       [   101,  29971,   1523, ...,  15484,  44588,    132],
       ...,
       [   101,  29971,   1523, ..., 116553,    132,    102],
       [   101,  29971,   1523, ...,  89408,   9665,   1471],
       [   101,  29971,   1523, ...,   7792,    852,  21640]]), label_ids=array([[  101, 26551, 10673, ...,     0,     0,     0],
       [  101, 26551, 10673, ...,     0,     0,     0],
       [  101, 26551, 10673, ..., 71672,  1757,   102],
       ...,
       [  101, 26551, 10673, ...,     0,     0,     0],
       [  101, 29971,  1523, ...,     0,     0,     0],
       [  101, 29971,  1523, ...,     0,     0,     0]]), metrics={'eval_loss': 1.5075504779815674, 'eval_rouge1': 80.3451, 'eval_rouge2': 1.3547, 'eval_rougeL': 80.3674, 'eval_rougeLsum': 80.3341, 'eval_gen_len': 47.6144, 'eval_runtime': 1367.4967, 'eval_samples_per_second': 1.963}

Теперь сгенерируем summary и посмотрим на результат случайных 10 примеров из тестовой выборки:

In [ ]:
small_test_dataset = test_dataset.select(range(10))

In [ ]:
model.to('cpu')

In [ ]:
pred_summaries = []

In [ ]:
for pair in small_test_dataset:
  text = pair['texts']
  target_summary = pair['summaries']
  inputs = tokenizer([text], max_length=512, return_tensors='pt')
  summary_ids = model.generate(inputs['input_ids'], max_length=60, early_stopping=True)
  pred_summary = ' '.join([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])
  pred_summaries.append(pred_summary)

In [ ]:
for i in range(10):
  print('------Original summary------:')
  print(small_test_dataset['summaries'][i])
  print('------Generated summary------:')
  print(pred_summaries[i])
  print('\n')

------Original summary------:
Рассматривается дело o взыскании суммы обеспечительного платежа, излишне уплаченной арендной платы в соответствии с соглашением о расторжении договора аренды нежилого помещения, процентов за пользование чужими денежными средствами.  Ответчиком обязательства по соглашению не исполнены.  В удовлетворении требования отказано, поскольку пропущен срок исковой давности.
------Generated summary------:
заявляется требование o взыскании обеспечительного платежа, излишне уплаченнои аренднои платы, процентов за пользование чужими денежными средствами. арендодатель ссылается на ненадлежащее исполнение арендатором обязательств по внесению обеспечительных платежеи. в удовлетворении требования отказано, поскольку арендатором не доказан факт ненадлежащего


------Original summary------:
Рассматривается дело o взыскании неустойки, процентов за пользование коммерческим кредитом.  В обоснование исковых требований истец указал, что в установленные государственным контрактом с